**AI & Machine Learning (KAN-CINTO4003U) - Copenhagen Business School | Spring 2025**

***


# Part III: LLM

Please see the description of the assignment in the README file (section 3) <br>
**Guide notebook**: [guides/llm_guide.ipynb](guides/llm_guide.ipynb)


***

<br>

* Note that you should report results using a classification report. 

* Also, remember to include some reflections on your results: how do they compare with the results from Part I, BoW?, and part II, BERT? Are there any hyperparameters or prompting techniques that are particularly important?

* You should follow the steps given in the `llm_guide` notebook

<br>


***

## Imports


In [156]:
# imports for the project

import pandas as pd
from sklearn.metrics import classification_report 
from tqdm import tqdm
from decouple import Config
from decouple import RepositoryEnv
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.foundation_models.schema import TextGenParameters

## Load APi key

In [157]:

# Create a Config instance with the path to the .env file
config = Config(RepositoryEnv('.env'))

	# Load the WX_API_KEY from the .env file
wx_api_key = config('KEY')



## Connecting to the WatsonX.ai Credentials API

In [158]:
credentials = Credentials(
    url = "https://us-south.ml.cloud.ibm.com",
    api_key = wx_api_key
)

client = APIClient(
    credentials=credentials, 
    project_id="52ba1404-06c0-47d5-87c2-df078023a34e"
)

## Test connection

In [159]:

model = ModelInference(
    api_client=client,
    model_id="google/flan-t5-xl",
)

### 1. Load the data

We can load this data directly from [Hugging Face Datasets](https://huggingface.co/docs/datasets/) - The HuggingFace Hub- into a Pandas DataFrame. Pretty neat!

**Note**: This cell will download the dataset and keep it in memory. If you run this cell multiple times, it will download the dataset multiple times.

You are welcome to increase the `frac` parameter to load more data.

In [160]:

splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
# train = pd.read_parquet("hf://datasets/fancyzhx/ag_news/" + splits["train"])
test = pd.read_parquet("hf://datasets/fancyzhx/ag_news/" + splits["test"])

In [161]:
label_map = {
    0: 'World',
    1: 'Sports',
    2: 'Business',
    3: 'Sci/Tech'
}

def preprocess(df: pd.DataFrame, frac = 1e-2, label_map = label_map, seed=42) -> pd.DataFrame:
    return  (
        df
        .assign(label=lambda x: x['label'].map(label_map))
        [lambda df: df['label'].isin(label_map.values())]
        .groupby('label')
        .apply(lambda x: x.sample(frac=frac, random_state=seed))
        .reset_index(drop=True)

    )

# train_df = preprocess(train, frac=0.01)
test_df = preprocess(test, frac=0.1)

# clear up some memory by deleting the original dataframes
# del train
del test

test_df.shape, # train_df.shape, 

test_df.head()

,text,label
0,"Ford: Monthly Sales Drop, Company Looks To New...",Business
1,United #39;s pension dilemma United Airlines s...,Business
2,Comcast part of group wanting to buy MGM A con...,Business
3,Treasuries Tussle with Profit-Takers NEW YORK...,Business
4,"Lloyds TSB to Move More Than 1,000 UK Jobs to ...",Business


## Parameters

In [ ]:
PARAMS = TextGenParameters(
    temperature=0,              
    max_new_tokens=10,          
    stop_sequences=[".", "\n"], # Stop generating text when these sequences are encountered
)

model = ModelInference(
    api_client=client,
    model_id="google/flan-t5-xl",  # Model ID of the model to use, this is the T5 model and the accuracy was the highest using this model. 
    params=PARAMS
)

## Prompt

In [163]:
SYSTEM_PROMPT = """You task is to classify news stories into one of four categories. 

CATEGORIES:
{categories}

TEXT:
{text}

Please assign the correct category to the text. Answer with the correct category and nothing else.

Category:
"""

## Predictions

In [164]:
CATEGORIES = "- " + "\n- ".join(test_df["label"].unique())  # Create a string with all categories
print(CATEGORIES)

predictions = []

for text in tqdm(test_df["text"]):

    # format the prompt with the categories and the text
    prompt = SYSTEM_PROMPT.format(categories=CATEGORIES, text=text)
    
    # generate the response from the model
    response = model.generate(prompt)

    # extract the generated text from the response
    prediction = response["results"][0]["generated_text"].strip()

    # append the prediction to the list of predictions
    predictions.append(prediction)

- Business
- Sci/Tech
- Sports
- World


100%|██████████| 760/760 [03:21<00:00,  3.77it/s]


In [165]:
test_df["prediction"] = predictions
test_df

,text,label,prediction
0,"Ford: Monthly Sales Drop, Company Looks To New...",Business,Business
1,United #39;s pension dilemma United Airlines s...,Business,Business
2,Comcast part of group wanting to buy MGM A con...,Business,Business
3,Treasuries Tussle with Profit-Takers NEW YORK...,Business,Business
4,"Lloyds TSB to Move More Than 1,000 UK Jobs to ...",Business,Business
...,...,...,...
755,Palestinian Attack Kills Woman in Gaza Settlem...,World,World
756,Mounties left in dark by U.S. on deportation o...,World,World
757,Yemeni Poet Says He Is al-Qaida Member GUANTAN...,World,World
758,Saudis Take a Small Dose of Democracy For the ...,World,World


## Evaluation

In [166]:
print(classification_report(test_df.label, predictions))

              precision    recall  f1-score   support

    Business       0.90      0.88      0.89       190
    Sci/Tech       0.93      0.86      0.89       190
      Sports       0.98      0.97      0.98       190
       World       0.87      0.95      0.91       190

    accuracy                           0.92       760
   macro avg       0.92      0.92      0.92       760
weighted avg       0.92      0.92      0.92       760

